In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('clean_data.csv')

In [3]:
df.head()

,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,arrival_date_quarter,total_orang,days,lead_time_special_requests_ratio,cancellation_ratio,guest_night_interaction
0,0,Resort Hotel,0,13,2015,July,27,1,0,1,...,0,Transient,0,0,Quarter_3,1,1,13.0,0.0,1
1,1,Resort Hotel,0,14,2015,July,27,1,0,2,...,0,Transient,0,1,Quarter_3,2,2,7.0,0.0,4
2,2,Resort Hotel,0,9,2015,July,27,1,0,2,...,0,Transient,0,1,Quarter_3,2,2,4.5,0.0,4
3,3,Resort Hotel,1,85,2015,July,27,1,0,3,...,0,Transient,0,1,Quarter_3,2,3,42.5,0.0,6
4,4,Resort Hotel,1,75,2015,July,27,1,0,3,...,0,Transient,0,0,Quarter_3,2,3,75.0,0.0,6


In [4]:
from NaiveBayes import NaiveBayes

In [5]:
selected_features = [
    'is_canceled',
    'hotel',
    'stays_in_weekend_nights',
    'adults',
    'children',
    'market_segment',
    'total_of_special_requests',
    'is_repeated_guest',
    'previous_cancellations',
    'previous_bookings_not_canceled',
    'deposit_type',
    'total_orang'
]

df = df[selected_features]

In [6]:
df.head()

,is_canceled,hotel,stays_in_weekend_nights,adults,children,market_segment,total_of_special_requests,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,deposit_type,total_orang
0,0,Resort Hotel,0,1,0,Corporate,0,0,0,0,No Deposit,1
1,0,Resort Hotel,0,2,0,Online TA,1,0,0,0,No Deposit,2
2,0,Resort Hotel,0,2,0,Direct,1,0,0,0,No Deposit,2
3,1,Resort Hotel,0,2,0,Online TA,1,0,0,0,No Deposit,2
4,1,Resort Hotel,0,2,0,Offline TA/TO,0,0,0,0,No Deposit,2


In [7]:
df = pd.get_dummies(df, columns=['deposit_type'])
df = pd.get_dummies(df, columns=['hotel'])
df = pd.get_dummies(df, columns=['market_segment'])
df.head()

,is_canceled,stays_in_weekend_nights,adults,children,total_of_special_requests,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,total_orang,deposit_type_No Deposit,...,deposit_type_Refundable,hotel_City Hotel,hotel_Resort Hotel,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA
0,0,0,1,0,0,0,0,0,1,True,...,False,False,True,False,False,True,False,False,False,False
1,0,0,2,0,1,0,0,0,2,True,...,False,False,True,False,False,False,False,False,False,True
2,0,0,2,0,1,0,0,0,2,True,...,False,False,True,False,False,False,True,False,False,False
3,1,0,2,0,1,0,0,0,2,True,...,False,False,True,False,False,False,False,False,False,True
4,1,0,2,0,0,0,0,0,2,True,...,False,False,True,False,False,False,False,False,True,False


In [8]:
df = df.astype(int)
df.dtypes

is_canceled                       int32
stays_in_weekend_nights           int32
adults                            int32
children                          int32
total_of_special_requests         int32
is_repeated_guest                 int32
previous_cancellations            int32
previous_bookings_not_canceled    int32
total_orang                       int32
deposit_type_No Deposit           int32
deposit_type_Non Refund           int32
deposit_type_Refundable           int32
hotel_City Hotel                  int32
hotel_Resort Hotel                int32
market_segment_Aviation           int32
market_segment_Complementary      int32
market_segment_Corporate          int32
market_segment_Direct             int32
market_segment_Groups             int32
market_segment_Offline TA/TO      int32
market_segment_Online TA          int32
dtype: object

In [9]:
df.corr()['is_canceled'].sort_values(ascending=False)

is_canceled                       1.000000
market_segment_Online TA          0.181537
deposit_type_Non Refund           0.138105
total_orang                       0.077741
adults                            0.062883
hotel_City Hotel                  0.061073
previous_cancellations            0.059429
children                          0.053068
stays_in_weekend_nights           0.032081
deposit_type_Refundable           0.013675
market_segment_Aviation          -0.002907
market_segment_Groups            -0.006551
market_segment_Complementary     -0.013314
market_segment_Corporate         -0.019435
previous_bookings_not_canceled   -0.040323
is_repeated_guest                -0.051505
hotel_Resort Hotel               -0.061073
market_segment_Direct            -0.080826
deposit_type_No Deposit          -0.138136
total_of_special_requests        -0.141692
market_segment_Offline TA/TO     -0.152313
Name: is_canceled, dtype: float64

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Contoh data frame
# Misalkan df memiliki kolom 'feature1', 'feature2', ..., 'featureN' dan 'target' untuk y.
# df = pd.DataFrame({...})

# Pisahkan fitur (X) dan target (y)
X, y = df.drop(columns=['is_canceled']).values, df['is_canceled'].values

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat model Naive Bayes
model = GaussianNB()

# Melatih model
model.fit(X_train, y_train)

# Membuat prediksi pada data uji
y_pred = model.predict(X_test)

# Mengukur akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi model Naive Bayes: {accuracy:.2f}")

Akurasi model Naive Bayes: 0.72


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X, y = df.drop(columns=['is_canceled']).values, df['is_canceled'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

nb = NaiveBayes()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

X_train shape: (55931, 20)
X_test shape: (13983, 20)
y_train shape: (55931,)
y_test shape: (13983,)


f:\pindahan\S2\AI Lanjut\Tubes\Tubes2\Tugas-Besar-2-IF5150\NaiveBayes.py:35: RuntimeWarning: divide by zero encountered in log
  posterior = np.sum(np.log(self._pdf(idx, x)))


Accuracy: 0.7224486876921977

Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.96      0.83      9905
           1       0.60      0.14      0.23      4078

    accuracy                           0.72     13983
   macro avg       0.67      0.55      0.53     13983
weighted avg       0.69      0.72      0.66     13983

